In [9]:
!pip install gradio

In [14]:
pip install gradio google-generativeai

In [31]:
import gradio as gr
import google.generativeai as genai

# Configure Gemini API
GENAI_API_KEY = "your_api_key"  # Replace with your actual Gemini API key
genai.configure(api_key=GENAI_API_KEY)

# Predefined technical questions (used as a backup if AI fails)
role_questions = {
    "AI/ML Intern": {
        "Fresher": [
            "What is the difference between supervised and unsupervised learning?",
            "Explain a machine learning project you worked on during your academics.",
            "What are the key steps in building a machine learning model?",
            "How does gradient descent work in neural networks?",
            "What is the role of activation functions in deep learning?"
        ],
        "Experienced": [
            "How do you optimize hyperparameters in deep learning models?",
            "Explain the concept of transfer learning and its applications.",
            "How have you applied machine learning in real-world projects?",
            "What are some challenges in deploying ML models at scale?",
            "How do you handle imbalanced datasets in classification problems?"
        ]
    },
    "QA Testing": {
        "Fresher": [
            "What is the difference between functional and non-functional testing?",
            "What are different types of software testing methodologies?",
            "Explain the importance of test cases and how to design them.",
            "How would you test a login functionality?",
            "What is the role of regression testing?"
        ],
        "Experienced": [
            "How do you design an effective test automation framework?",
            "Explain your experience in API testing and tools you have used.",
            "How do you handle debugging a failing test case?",
            "What are the best practices for writing test scripts?",
            "How do you ensure test coverage in an Agile environment?"
        ]
    }
}

# Function to generate AI-based questions
def generate_ai_questions(position, fresher_experienced, tech_stack):
    prompt = f"""
    You are an interviewer preparing a technical interview for a {fresher_experienced} candidate applying for {position}.
    The candidate has the following tech skills: {tech_stack}.
    Generate 5 **technical** interview questions that are role-specific and relevant to the experience level.
    Questions should be clear, challenging, and related to real-world applications.
    """

    try:
        response = genai.generate_content(
            prompt,
            model="gemini-pro",
            temperature=0.5,
            max_tokens=250
        )

        questions = response.text.split("\n")
        filtered_questions = [q.strip("- ") for q in questions if q.strip() != ""]

        return filtered_questions[:5]  # Return up to 5 questions
    except Exception as e:
        return ["⚠️ AI Question Generation Failed. Using Predefined Questions."]

# Function to generate final questions (fallback to predefined if AI fails)
def generate_final_questions(position, fresher_experienced, tech_stack):
    ai_questions = generate_ai_questions(position, fresher_experienced, tech_stack)
    predefined_questions = role_questions.get(position, {}).get(fresher_experienced, ["No predefined questions available."])

    # Select AI-generated questions if available; otherwise, use predefined
    if ai_questions[0] != "⚠️ AI Question Generation Failed. Using Predefined Questions.":
        final_questions = ai_questions
    else:
        final_questions = predefined_questions

    # Add non-technical questions
    final_questions.append("Describe your previous experience in this field (if any).")
    final_questions.append("Why are you interested in this role?")

    return final_questions[:7]  # Ensure exactly 7 questions

# Function to handle candidate input
def hiring_assistant(name, email, phone, gender, grad_year, grad_field, specialization, college, experience, fresher_experienced, position, location, tech_stack):
    if not name or not email or not position:
        return "⚠️ Please fill in all required fields.", []

    # Generate questions
    questions = generate_final_questions(position, fresher_experienced, tech_stack)

    return (
        f"Thank you, {name}! You have applied for **{position}**.\n\n"
        f"📍 Location: {location}\n"
        f"🎓 College: {college} ({grad_year}, {specialization})\n"
        f"👨‍🎓 Experience Level: {fresher_experienced}\n"
        f"🛠️ Tech Stack: {tech_stack}\n\n"
        f"Please answer the following questions:", questions
    )

# Function to submit responses
def submit_application(*answers):
    return "✅ Thank you for your responses! If you are shortlisted, our hiring team will contact you soon."

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("# 🚀 TalentScout - AI Hiring Assistant")
    gr.Markdown("Welcome! This chatbot will guide you through the hiring process. Please provide the required details below.")

    with gr.Row():
        name = gr.Textbox(label="Full Name")
        email = gr.Textbox(label="Email")
        phone = gr.Textbox(label="Phone Number")

    with gr.Row():
        gender = gr.Dropdown(label="Gender", choices=["Male", "Female", "Other", "Prefer not to say"])
        grad_year = gr.Number(label="Graduation Year", value=2024)

    with gr.Row():
        grad_field = gr.Dropdown(
            label="Graduation Field",
            choices=["B.Tech", "M.Tech", "B.Sc", "M.Sc", "BCA", "MCA", "Other"],
            value="B.Tech"
        )
        specialization = gr.Dropdown(
            label="Specialization",
            choices=["CSE", "IT", "AI/ML", "AIDS", "ECE", "EEE", "Mech", "Civil", "Other"],
            value="CSE"
        )

    college = gr.Textbox(label="College Name")

    with gr.Row():
        experience = gr.Number(label="Years of Experience", value=0)
        fresher_experienced = gr.Dropdown(
            label="Fresher or Experienced?",
            choices=["Fresher", "Experienced"],
            value="Fresher"
        )

    tech_stack = gr.Textbox(label="Tech Stack (e.g., Python, TensorFlow, SQL)", placeholder="Enter your technology skills")

    with gr.Row():
        position = gr.Dropdown(
            label="Desired Position",
            choices=list(role_questions.keys()),  # Dropdown options based on predefined roles
            value="AI/ML Intern"
        )
        location = gr.Textbox(label="Current Location")

    generate_btn = gr.Button("🔍 Generate Questions")
    output_text = gr.Textbox(label="Message", interactive=False)
    output_questions = gr.List(label="Technical Questions")

    generate_btn.click(
        hiring_assistant,
        inputs=[name, email, phone, gender, grad_year, grad_field, specialization, college, experience, fresher_experienced, position, location, tech_stack],
        outputs=[output_text, output_questions]
    )

    answer_boxes = [gr.Textbox(label=f"Answer {i+1}") for i in range(7)]

    submit_btn = gr.Button("✅ Submit Application")
    final_message = gr.Textbox(label="Final Message", interactive=False)

    submit_btn.click(
        submit_application,
        inputs=answer_boxes,
        outputs=final_message
    )

# Launch App
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3ffa99db6e505afc3c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [35]:
import gradio as gr

# Predefined technical questions based on roles (Separate for Freshers and Experienced)
role_questions = {
    "AI/ML Intern": {
        "Fresher": [
            "What is the difference between supervised and unsupervised learning?",
            "Why are you interested in AI/ML?",
            "Explain a machine learning project you worked on during your academics."
        ],
        "Experienced": [
            "How do you optimize hyperparameters in deep learning models?",
            "Explain the concept of transfer learning and its applications.",
            "How have you applied machine learning in real-world projects?"
        ]
    },
    "QA Testing": {
        "Fresher": [
            "What is the difference between functional and non-functional testing?",
            "Why do you want to build a career in QA Testing?",
            "What are different types of software testing methodologies?"
        ],
        "Experienced": [
            "How do you design an effective test automation framework?",
            "Explain your experience in API testing and tools you have used.",
            "How do you handle debugging a failing test case?"
        ]
    },
    "SDE": {
        "Fresher": [
            "What are the key principles of object-oriented programming?",
            "Why do you want to work as a Software Developer?",
            "Explain a coding project you've worked on."
        ],
        "Experienced": [
            "How do you optimize SQL queries for performance?",
            "Explain time and space complexity in real-world scenarios.",
            "Have you worked with microservices? If yes, explain the architecture."
        ]
    },
    "Full Stack Developer": {
        "Fresher": [
            "What is the difference between REST and GraphQL?",
            "Why do you want to be a Full Stack Developer?",
            "Explain the MVC architecture in web development."
        ],
        "Experienced": [
            "How do you ensure the security of a web application?",
            "Explain the importance of state management in front-end development.",
            "Describe a full-stack project you built and the technologies used."
        ]
    },
    "Project Manager": {
        "Fresher": [
            "What do you know about Agile methodology?",
            "Why are you interested in project management?",
            "How would you handle a conflict between two team members?"
        ],
        "Experienced": [
            "What strategies do you use for project risk management?",
            "Explain a challenging project you managed and how you handled it.",
            "How do you ensure effective communication within a team?"
        ]
    },
    "Business Development Associate (BDA)": {
        "Fresher": [
            "What are the key skills required for a Business Development Associate?",
            "Why do you want to work in business development?",
            "How would you approach a new client for partnership?"
        ],
        "Experienced": [
            "Explain a sales funnel and how it works.",
            "How do you handle client objections effectively?",
            "Describe your experience in market research and lead generation."
        ]
    },
    "SDE Intern": {
        "Fresher": [
            "What is the difference between an array and a linked list?",
            "Explain recursion with an example.",
            "What are the common sorting algorithms and their time complexities?"
        ],
        "Experienced": [
            "How do you write efficient and scalable code?",
            "Explain your experience with multi-threading in programming.",
            "Describe an optimization technique you used in a project."
        ]
    }
}

# Function to generate questions
def hiring_assistant(name, email, phone, gender, grad_year, grad_field, specialization, college, experience, fresher_experienced, position, location):
    if not name or not email or not position:
        return "⚠️ Please fill in all required fields.", []

    # Get predefined questions based on role and experience level
    questions = role_questions.get(position, {}).get(fresher_experienced, ["No predefined questions available for this role."])

    return (
        f"Thank you, {name}! You have applied for **{position}**.\n\n"
        f"📍 Location: {location}\n"
        f"🎓 College: {college} ({grad_year}, {specialization})\n"
        f"👨‍🎓 Experience Level: {fresher_experienced}\n\n"
        f"Please answer the following questions:", questions
    )

# Function to submit responses
def submit_application(*answers):
    return "✅ Thank you for your responses! If you are shortlisted, our hiring team will contact you soon."

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("# 🚀 TalentScout - AI Hiring Assistant")

    with gr.Row():
        name = gr.Textbox(label="Full Name")
        email = gr.Textbox(label="Email")
        phone = gr.Textbox(label="Phone Number")

    with gr.Row():
        gender = gr.Dropdown(label="Gender", choices=["Male", "Female", "Other", "Prefer not to say"])
        grad_year = gr.Number(label="Graduation Year", value=2024)

    with gr.Row():
        grad_field = gr.Dropdown(
            label="Graduation Field",
            choices=["B.Tech", "M.Tech", "B.Sc", "M.Sc", "BCA", "MCA", "Other"],
            value="B.Tech"
        )
        specialization = gr.Dropdown(
            label="Specialization",
            choices=["CSE", "AI/ML", "AIDS", "ECE", "EEE", "Mech", "Civil", "Other"],
            value="CSE"
        )

    college = gr.Textbox(label="College Name")

    with gr.Row():
        experience = gr.Number(label="Years of Experience", value=0)
        fresher_experienced = gr.Dropdown(
            label="Fresher or Experienced?",
            choices=["Fresher", "Experienced"],
            value="Fresher"
        )

    with gr.Row():
        position = gr.Dropdown(
            label="Desired Position",
            choices=list(role_questions.keys()),  # Dropdown options based on predefined roles
            value="AI/ML Intern"  # Default selection
        )
        location = gr.Textbox(label="Current Location")

    generate_btn = gr.Button("🔍 Generate Questions")
    output_text = gr.Textbox(label="Message", interactive=False)
    output_questions = gr.List(label="Technical Questions")

    generate_btn.click(
        hiring_assistant,
        inputs=[name, email, phone, gender, grad_year, grad_field, specialization, college, experience, fresher_experienced, position, location],
        outputs=[output_text, output_questions]
    )

    answer_boxes = [gr.Textbox(label=f"Answer {i+1}") for i in range(3)]

    submit_btn = gr.Button("✅ Submit Application")
    final_message = gr.Textbox(label="Final Message", interactive=False)

    submit_btn.click(
        submit_application,
        inputs=answer_boxes,
        outputs=final_message
    )

# Launch App
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bd0c313f9e22c3bf8e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
